# Create Transit Rich Areas Dataset

In [1]:
import os
import sys
import pandas as pd, geopandas as gp, matplotlib.pyplot as plt, numpy as np
import json
import fiona
import requests
import getpass
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *

### Read in password from file

**Important:** Do not put passwords or sensitive information into this notebook. Read in from seperate file or add to DataViz creds file. 

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [3]:
pass_txt = '../../../../passwords/arcgis_enterprise_pass.txt'
password = get_file_contents(pass_txt)

In [4]:
gis = GIS("https://arcgis.ad.mtc.ca.gov/portal/", "jcroff", password)

## Pull Existing Transit Stops 2020 Data from Portal

In [5]:
#Pull data from arcgis portal
transit_stops_exist = pull_geodata_from_argis('3faf8401623b48ae8d70f7a71d7365c9',client=gis)

In [6]:
#project to NAD83 UTMZ10
transit_stops_exist_nad83 = transit_stops_exist.to_crs('EPSG:26910')

/Users/jcroff/anaconda3/envs/geo_env/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
transit_stops_exist_nad83.crs

'EPSG:26910'

In [15]:
transit_stops_exist_nad83.head(5)

,agency_id,agency_nm,hdwy_15min,hdwy_30min,hdwy_class,loc_type,major_stop,objectid,route_ty_t,route_type,stop_code,stop_desc,stop_id,stop_name,geom
0,BA,Bay Area Rapid Transit,1,1,15 mins or less,0,1,1,Rail,2.0,12TH,,12TH,12th St. Oakland City Center,POINT (564135.244 4184293.205)
1,BA,Bay Area Rapid Transit,1,1,15 mins or less,0,1,2,Rail,2.0,16TH,,16TH,16th St. Mission,POINT (551112.002 4179906.983)
2,BA,Bay Area Rapid Transit,1,1,15 mins or less,0,1,3,Rail,2.0,19TH,,19TH,19th St. Oakland,POINT (564382.160 4184803.318)
3,BA,Bay Area Rapid Transit,1,1,15 mins or less,0,1,4,Rail,2.0,24TH,,24TH,24th St. Mission,POINT (551257.209 4178510.984)
4,BA,Bay Area Rapid Transit,1,1,15 mins or less,0,1,5,Rail,2.0,ANTC,,ANTC,Antioch,POINT (607084.546 4206005.178)


In [8]:
transit_stops_exist_nad83['route_ty_t'].unique()

array(['Rail', 'Bus', 'Tram, Streetcar, Light Rail', 'Cable Tram', None,
       'Ferry'], dtype=object)

In [9]:
#Filter stops to include stops w/ types as well as stops with routes headways of 15 mins or less
transit_stops_exist_nad83 = transit_stops_exist_nad83[(transit_stops_exist_nad83['hdwy_15min'] == 1) & 
                                                  (transit_stops_exist_nad83['route_ty_t'].isin(['Rail', 
                                                                                              'Bus', 
                                                                                              'Tram, Streetcar, Light Rail',
                                                                                              'Cable Tram',
                                                                                              'Ferry']))]

In [10]:
transit_stops_exist_nad83.shape

(6414, 15)

# Pull planned and potential stops from portal

In [12]:
#Pull data from arcgis portal
transit_stops_plan = pull_geodata_from_argis('18a6239819b048fab9c87bb4d7649560',client=gis)

In [13]:
#Project to NAD83 UTMZ10
transit_stops_plan_nad83 = transit_stops_plan.to_crs('EPSG:26910')

/Users/jcroff/anaconda3/envs/geo_env/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
transit_stops_plan_nad83.crs

'EPSG:26910'

In [16]:
transit_stops_plan_nad83.head(5)

,objectid,county,jurisdictn,agency_nm,stop_name,route_ty_t,ppa_id,ppa_name,committd,status,exp_open,geom
0,1,CA013,Brentwood,TBD,Brentwood,Bus,2202,Bus Rapid Transit to Brentwood (formerly BART ...,No,Final Blueprint,2021-2035,POINT (610284.281 4201385.715)
1,2,CA097,Healdsburg,Sonoma-Marin Area Rail Transit,Healdsburg,Rail,2304,SMART to Cloverdale,No,Not Included,Not Included,POINT (511636.968 4273185.826)
2,3,CA097,Cloverdale,Sonoma-Marin Area Rail Transit,Cloverdale,Rail,2304,SMART to Cloverdale,No,Not Included,Not Included,POINT (498953.967 4294420.424)
3,4,CA085,San Jose,Santa Clara Valley Transportation Authority,San Jose Airport,"Tram, Streetcar, Light Rail",2402,San Jose Airport People Mover,No,Not Included,Not Included,POINT (595075.482 4136054.501)
4,5,CA085,Cupertino,Bay Area Rapid Transit,De Anza College,"Tram, Streetcar, Light Rail",2206,Stevens Creek Line (formerly BART to Cupertino),No,Final Blueprint,2036-2050,POINT (585044.618 4131116.515)


In [19]:
transit_stops_plan_nad83['status'].unique()

array(['Final Blueprint', 'Not Included', 'Draft Blueprint',
       'Under Construction'], dtype=object)

In [20]:
#filter for stops in the Final Blueprint or Under Construction
transit_stops_plan_nad83 = transit_stops_plan_nad83[transit_stops_plan_nad83['status'].isin(['Final Blueprint',
                                                                                             'Under Construction'])]

In [ ]:
#Pull PBA2050 transit projects 